In [ ]:
from flask import Flask,  render_template, request, jsonify, redirect, url_for, flash
import pymysql
import hashlib

app = Flask(__name__)

app.secret_key = hashlib.sha256(b"supersecretkey").hexdigest()

conn = pymysql.connect(host="127.0.0.1", user = "root", password = "mysql904#", db = "myDB", charset = "utf8")
cur = conn.cursor()

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/login")
def login():
    return render_template("login.html")

@app.route("/logincheck", methods = ["post"])
def logincheck():
    # 회원인지 아닌지 여부를 처리하는 부분
    idu = request.form["id"]
    pwdu = request.form["pwd"]

    try:
        cur.execute("SELECT id, pwd FROM usertable2 WHERE id=%s", (idu,))
        result = cur.fetchone()

        if result:  # 아이디가 존재하는 경우
            if result[1] == pwdu:  # 비밀번호가 일치하는 경우
                return render_template("membermain.html")
            else:
                flash("비밀번호가 틀렸습니다.")  # 비밀번호가 틀리면 알림
                return redirect(url_for("login"))
        else:
            flash("존재하지 않는 아이디입니다.")  # 아이디가 없는 경우
            return redirect(url_for("login"))
    except Exception as e:
        # 예외 처리: 로그 파일에 오류를 기록하거나 다른 방법으로 오류를 처리
        flash("서버 오류가 발생했습니다. 나중에 다시 시도해 주세요.")
        print(f"Error: {e}")  # 터미널에서 오류 확인
        return redirect(url_for("login"))


@app.route("/join")
def joinApp():
    return render_template("join.html")


@app.route("/idcheck", methods=["POST"])
def idcheck():
    idu = request.form["id"]
    print(f"Checking ID: {idu}")  # Debugging log to see the incoming ID

    cur.execute("SELECT id FROM usertable2 WHERE id = %s", (idu,))
    result = cur.fetchone()

    if result:
        return jsonify({"status": "exists"})
    else:
        return jsonify({"status": "available"})

# @app.route("/logincheck", methods = ["post"])
# def logincheck():
#     idu = request.form["id"]
#     pwdu = request.form["pwd"]



@app.route("/memberm")
def membermain():
    return render_template("membermain.html")


@app.route("/insert", methods=['post'])
def insertApp():
    id = request.form['id']
    pwd = request.form['pwd']
    # pwdsec = hashlib.sha256(pwd.encode("utf-8")).hexdigest()
    name = request.form['name']    
    birthy = request.form['birthy']
    birthm = request.form['birthm']
    birthd = request.form['birthd']
    gender = request.form['gender']
    tel = request.form['tel']
    post = request.form['post']
    addr = request.form['addr']
    addrdt = request.form['addrdt']
    email = request.form['email']
    cur.execute("insert into usertable2 values('%s','%s','%s',%d,%d,%d,'%s','%s',%d,'%s','%s','%s')"
        %(id,pwd,name,int(birthy),int(birthm),int(birthd),gender,tel,int(post),addr,addrdt,email))
    conn.commit()
    return render_template('login.html')


host_addr = "127.0.0.1"
port_num = "5000"

if __name__ == "__main__":
    app.run(host = host_addr, port = port_num)